In [41]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field

from typing import TypedDict, Annotated

import os
from dotenv import load_dotenv

import operator

In [42]:
class EvaluationSchema(BaseModel):
    feedback: str= Field(description="Detail feedback for the essay")
    score: int= Field(description="Score out of 10", ge=0, le= 10)
    

In [43]:
load_dotenv()

llm= HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task= "text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_ENDPOINT"),
    max_new_tokens= 500,
    temperature= 0.3
)

model= ChatHuggingFace(llm= llm)
# parser= PydanticOutputParser(pydantic_object=EvaluationSchema)

# prompt = PromptTemplate(
#     template="Evaluate the following essay.\n{format_instructions}\n\nEssay: {essay}\n",
#     input_variables=["essay"],
#     partial_variables={"format_instructions": parser.get_format_instructions()}
# )

# chain=  model | parser

In [44]:
def modelDefine(promptTemplate: str):
    llm= HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task= "text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_ENDPOINT"),
    max_new_tokens= 500,
    temperature= 0.3
    )

    model= ChatHuggingFace(llm= llm)
    parser= PydanticOutputParser(pydantic_object=EvaluationSchema)

    prompt = PromptTemplate(
        # template="{prompt}.\nThe output should be in the given format: {format_instructions}\n",
        template="""You are an essay evaluator. Analyze the following essay and provide feedback.

                    {prompt}

                    You must respond with a valid JSON object (not a schema definition) with these exact fields:
                    - feedback: string with detailed analysis
                    - score: integer from 0 to 10

                    {format_instructions}

                    Remember: Provide actual values, not a schema definition.
                """,
        input_variables= ["prompt"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    chain= prompt | model | parser
    result= chain.invoke({"prompt": promptTemplate})
    return result

In [45]:
class UPSCState(TypedDict):
    essay: str
    
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]# add is the reducer
    avg_score: float
    

In [46]:
def evaluate_language(state: UPSCState):
    prompt=f"Evaluate the language qualify of the following essay and provide a feeedback and assign a score out of 10 |n {state['essay']}"
    output= modelDefine(prompt)
    
    return {'language_feedback': output.feedback, 'individual_scores':[output.score]}

In [47]:
def evaluate_analysis(state: UPSCState):
    prompt=f"Evaluate the depth of the following essay and provide a feeedback and assign a score out of 10 |n {state['essay']}"
    output= modelDefine(prompt)
    
    return {'analysis_feedback': output.feedback, 'individual_scores':[output.score]}

In [48]:
def evaluate_thought(state: UPSCState):
    prompt=f"Evaluate the clairty of the thought of the following essay and provide a feeedback and assign a score out of 10 |n {state['essay']}"
    output= modelDefine(prompt)
    
    return {'clarity_feedback': output.feedback, 'individual_scores':[output.score]}

In [49]:
def final_evaluation(state: UPSCState):
    # summary feedback
    prompt= f"Based on the following feedbacks create a summarized feedback \n language feedback: {state['language_feedback']} \n dept of the analysis feedback: {state['analysis_feedback']} \n clarity of thought feedback:{state['clarity_feedback']}"
    overall_feedback= model.invoke(prompt).content
    
    # avg calculate
    avg_score= sum(state['individual_scores'])/ len(state['individual_scores'])
    
    return {
        'overall_feedback': overall_feedback,
        'avg_score': avg_score
    }
    

In [50]:
graph= StateGraph(UPSCState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.add_edge('final_evaluation', END)

workflow= graph.compile()

In [51]:
essay="""
Gaming has evolved from a niche hobby into one of the most influential cultural and technological phenomena of the 21st century. What began with simple pixelated adventures like Pong and Tetris has transformed into immersive digital worlds powered by complex graphics, storytelling, and online communities. Today, gaming is not just entertainment—it’s an ecosystem blending art, technology, psychology, and even economics.

One of gaming’s most powerful effects lies in its interactivity. Unlike passive media, games place players inside the story, demanding problem-solving, strategy, and creativity. Competitive games like Valorant or League of Legends sharpen reflexes and teamwork, while narrative experiences such as The Last of Us explore deep emotional and moral questions. In essence, games have become virtual laboratories for human behavior and learning.

The rise of esports and streaming platforms has also turned gaming into a legitimate profession. Players, commentators, and developers now form an industry worth hundreds of billions of dollars. However, this growth has brought challenges—addiction, toxicity, and exploitative monetization models continue to plague the community. Responsible gaming habits and ethical design practices are crucial for keeping this digital playground healthy.

Beyond competition and profit, gaming has an unexpected social power. Online multiplayer games connect people across cultures, languages, and continents, often creating friendships that transcend geography. From virtual concerts in Fortnite to educational simulations used in classrooms, gaming continues to redefine how humans interact, learn, and create.

In short, gaming is no longer “just playing.” It’s storytelling, skill-building, and social connection wrapped in code and imagination—a reflection of how technology and creativity merge to shape the future of human experience.
"""

In [52]:
essay2= """
Gaming is something that many people do these days and it has become very big in the world. When people are playing games, they are doing it for fun, but also sometimes not for fun because it can make you angry too if you lose or the level is hard. Games are like movies but not really because you can play them and make your own story, but also sometimes the story is already made and you just go with it, which can be confusing.

There are many types of games like shooting games, racing games, and the other ones that you can play on phone or computer or also television. People play them to pass time, and also to talk to others on internet but sometimes it is not good because too much gaming can spoil the mind or eyes, and parents say not to do that but kids still do it because they enjoy a lot.

Some people even make money from gaming but it is not easy and you must practice very much and sometimes luck is also needed. There are games that teach things too, but not all of them. Some are just wasting of time and you get nothing from it except time gone.

In conclusion, gaming is nice but also not always good and people should do it sometimes but not too much because there are many other things in life too like studies and going outside and other work. Gaming is important but not everything and people should remember that before it becomes too much.
"""

In [58]:
initial_state= {
    'essay': essay2
}

final_state= workflow.invoke(initial_state)

In [59]:
final_state

{'essay': '\nGaming is something that many people do these days and it has become very big in the world. When people are playing games, they are doing it for fun, but also sometimes not for fun because it can make you angry too if you lose or the level is hard. Games are like movies but not really because you can play them and make your own story, but also sometimes the story is already made and you just go with it, which can be confusing.\n\nThere are many types of games like shooting games, racing games, and the other ones that you can play on phone or computer or also television. People play them to pass time, and also to talk to others on internet but sometimes it is not good because too much gaming can spoil the mind or eyes, and parents say not to do that but kids still do it because they enjoy a lot.\n\nSome people even make money from gaming but it is not easy and you must practice very much and sometimes luck is also needed. There are games that teach things too, but not all o